In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load T5 model and tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

def zero_shot_sentiment_analysis(text, sentiment_label):
    # Preprocess input text with sentiment label
    input_text = sentiment_label + " " + text

    # Tokenize input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Generate output
    output = model.generate(input_ids)

    # Decode output tokens
    predicted_sentiment = tokenizer.decode(output[0], skip_special_tokens=True)

    return predicted_sentiment

# Example usage
input_text = "I really enjoyed the movie, it was fantastic!"
sentiment_label = "positive:"
predicted_sentiment = zero_shot_sentiment_analysis(input_text, sentiment_label)
print("Predicted Sentiment:", predicted_sentiment)

ImportError: 
T5Tokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load T5 model and tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

def zero_shot_sentiment_analysis(text, sentiment_label):
    # Preprocess input text with sentiment label
    input_text = sentiment_label + " " + text

    # Tokenize input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Generate output
    output = model.generate(input_ids)

    # Decode output tokens
    predicted_sentiment = tokenizer.decode(output[0], skip_special_tokens=True)

    return predicted_sentiment

# Example usage
input_text = "I really enjoyed the movie, it was fantastic!"
sentiment_label = "positive:"
predicted_sentiment = zero_shot_sentiment_analysis(input_text, sentiment_label)
print("Predicted Sentiment:", predicted_sentiment)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Predicted Sentiment: : I really enjoyed the movie, it was fantastic!


In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load T5 model and tokenizer
tokenizer = T5Tokenizer.from_pretrained("t5-base")
model = T5ForConditionalGeneration.from_pretrained("t5-base")

def zero_shot_ner(text):
    # Prefix input text with task identifier
    input_text = "ner: " + text

    # Tokenize input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Generate output
    output = model.generate(input_ids)

    # Decode output tokens
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract named entities
    named_entities = decoded_output.split(",")

    return named_entities

# Example usage
input_text = "Steve Jobs and Steve Wozniak founded Apple Inc. in 1976."
predicted_entities = zero_shot_ner(input_text)
print(predicted_entities)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


['Steve Jobs and Steve Wozniak founded Apple Inc. in 1976.']


In [5]:
from transformers import PalmTokenizer, PalmForConditionalGeneration

# Load Palm 2 model and tokenizer
tokenizer = PalmTokenizer.from_pretrained("fathom/palm-2-small")
model = PalmForConditionalGeneration.from_pretrained("fathom/palm-2-small")

def zero_shot_ner(text):
    # Preprocess input text
    input_text = "[NER] " + text

    # Tokenize input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Generate output
    output = model.generate(input_ids)

    # Decode output tokens
    decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)

    return decoded_output

# Example usage
input_text = "Steve Jobs and Steve Wozniak founded Apple Inc. in 1976."
predicted_entities = zero_shot_ner(input_text)
print(predicted_entities)


ImportError: cannot import name 'PalmTokenizer' from 'transformers' (/usr/local/lib/python3.10/dist-packages/transformers/__init__.py)

In [6]:
from transformers import BertTokenizer, BertForTokenClassification
import torch

# Load pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForTokenClassification.from_pretrained('bert-base-uncased')

def zero_shot_ner(text):
    # Tokenize input text
    tokenized_text = tokenizer.encode(text, add_special_tokens=True)

    # Convert token IDs to tensor
    input_ids = torch.tensor([tokenized_text])

    # Perform inference
    with torch.no_grad():
        outputs = model(input_ids)

    # Extract predicted labels (named entities)
    predicted_labels = torch.argmax(outputs[0], dim=2).squeeze().tolist()

    # Map token IDs back to tokens
    tokens = tokenizer.convert_ids_to_tokens(tokenized_text)

    # Postprocess to extract named entities
    named_entities = []
    current_entity = []
    for token, label_id in zip(tokens, predicted_labels):
        label = model.config.id2label[label_id]
        if label.startswith('B-'):
            if current_entity:
                named_entities.append(' '.join(current_entity))
                current_entity = []
            current_entity.append(token)
        elif label.startswith('I-'):
            current_entity.append(token)
        elif label == 'O':
            if current_entity:
                named_entities.append(' '.join(current_entity))
                current_entity = []

    # Add the last entity if any
    if current_entity:
        named_entities.append(' '.join(current_entity))

    return named_entities

# Example usage
input_text = "Steve Jobs and Steve Wozniak founded Apple Inc. in 1976."
predicted_entities = zero_shot_ner(input_text)
print(predicted_entities)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[]


In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
import torch

# Load T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
config = T5Config.from_pretrained("t5-small", num_labels=2)  # Assuming binary classification for NER
model = T5ForConditionalGeneration.from_pretrained("t5-small", config=config)

# Prepare training data (input text and target text pairs)
# Example: ["Steve Jobs founded Apple Inc.", "B-PER O B-ORG O"]

# Fine-tune T5 model on the labeled dataset
# Example: model.train(input_text, target_text)

# Example usage for inference
def few_shot_ner(input_text):
    prompt = "ner: " + input_text
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    outputs = model.generate(inputs, max_length=50)  # Adjust max_length as needed
    predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return predicted_text

# Example usage
input_text = "Steve Jobs and Steve Wozniak founded Apple Inc. in 1976."
predicted_entities = few_shot_ner(input_text)
print(predicted_entities)


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Steve Jobs and Steve Wozniak founded Apple Inc. in 1976.


In [8]:
from transformers import T5Tokenizer, T5ForConditionalGeneration


In [10]:
# model_name = "google/t5-base"
model_name = "t5-base"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
prompt = """Consider the following sentence: "<sentence to be analyzed>". Identify the named entities and their types (e.g., Person: Barack Obama, Location: New York City).

Output: Analyzed sentence with named entities bracketed and labeled (e.g., [Person: Barack Obama] visited [Location: New York City]).
"""

sentence = "Barack Obama, the former president of the United States, visited New York City last week."
final_prompt = prompt.replace("<sentence to be analyzed>", sentence)


In [12]:
outputs = model.generate(
    input_ids=tokenizer.encode(final_prompt, return_tensors="pt"),
    max_length=len(tokenizer.encode(final_prompt, return_tensors="pt")) + 50,
    num_beams=5,
)

decoded_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded_text)


True


In [13]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel


In [14]:
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [20]:
prompt = """Consider the following sentence: "<sentence to be analyzed>". Identify the named entities and their types (e.g., Person: Barack Obama, Location: New York City).

After the analysis, I will write a sentence starting with "**Answer:**" containing the bracketed and labeled named entities (e.g., Answer: [Person: Barack Obama] visited [Location: New York City] last week).

"""

sentence = "Barack Obama, the former president of the United States, visited New York City last week."
final_prompt = prompt.replace("<sentence to be analyzed>", sentence)


In [21]:
outputs = model.generate(
    input_ids=tokenizer.encode(final_prompt, return_tensors="pt"),
    max_length=len(tokenizer.encode(final_prompt, return_tensors="pt")) + 500,
    num_beams=2,  # Adjust num_beams for better quality (more beams require more computation)
)

decoded_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Consider the following sentence: "Barack Obama, the former president of the United States, visited New York City last week.". Identify the named entities and their types (e.g., Person: Barack Obama, Location: New York City).

After the analysis, I will write a sentence starting with "**Answer:**" containing the bracketed and labeled named entities (e.g., Answer: [Person: Barack Obama] visited [Location: New York City] last week).


I will then write a sentence starting with "**Answer:**" containing the bracketed and labeled named entities (e.g., Answer: [Person: Barack Obama] visited [Location: New York City] last week).

I will then write a sentence starting with "**Answer:**" containing the bracketed and labeled named entities (e.g., Answer: [Person: Barack Obama] visited [Location: New York City] last week).

I will then write a sentence starting with "**Answer:**" containing the bracketed and labeled named entities (e.g., Answer: [Person: Barack Obama] visited [Location: New York C

In [32]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline


# Load a pre-trained GPT2 model and a tokenizer
model_name = 'gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Create a zero-shot NER pipeline
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

# Provide the text you want to analyze and a list of possible entities
text = "John Smith went to Washington D.C. to visit the White House."
entities = ["PERSON", "LOCATION", "ORGANIZATION"]

# Perform NER
outputs = nlp(text)

# Print the results
print(outputs)

Some weights of GPT2ForTokenClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'entity': 'LABEL_1', 'score': 0.63554376, 'index': 0, 'word': 'ĠJohn', 'start': 0, 'end': 4}, {'entity': 'LABEL_1', 'score': 0.8474974, 'index': 1, 'word': 'ĠSmith', 'start': 4, 'end': 10}, {'entity': 'LABEL_0', 'score': 0.70159686, 'index': 2, 'word': 'Ġwent', 'start': 10, 'end': 15}, {'entity': 'LABEL_0', 'score': 0.5216955, 'index': 3, 'word': 'Ġto', 'start': 15, 'end': 18}, {'entity': 'LABEL_1', 'score': 0.6736028, 'index': 4, 'word': 'ĠWashington', 'start': 18, 'end': 29}, {'entity': 'LABEL_0', 'score': 0.5804254, 'index': 5, 'word': 'ĠD', 'start': 29, 'end': 31}, {'entity': 'LABEL_1', 'score': 0.95643485, 'index': 6, 'word': '.', 'start': 31, 'end': 32}, {'entity': 'LABEL_0', 'score': 0.7367034, 'index': 7, 'word': 'C', 'start': 32, 'end': 33}, {'entity': 'LABEL_0', 'score': 0.5089512, 'index': 8, 'word': '.', 'start': 33, 'end': 34}, {'entity': 'LABEL_1', 'score': 0.756666, 'index': 9, 'word': 'Ġto', 'start': 34, 'end': 37}, {'entity': 'LABEL_1', 'score': 0.5112295, 'index': 1

In [36]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline


# Load a pre-trained t5-small model and a tokenizer
model_name = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Create a zero-shot NER pipeline
#nlp = pipeline("zero-shot-ner", model=model, tokenizer=tokenizer)
"""
KeyError: "Unknown task zero-shot-ner, available tasks are ['audio-classification', 'automatic-speech-recognition', 'conversational', 'depth-estimation', 'document-question-answering', 'feature-extraction', 'fill-mask', 'image-classification', 'image-feature-extraction', 'image-segmentation', 'image-to-image', 'image-to-text', 'mask-generation', 'ner', 'object-detection', 'question-answering', 'sentiment-analysis', 'summarization', 'table-question-answering', 'text-classification', 'text-generation', 'text-to-audio', 'text-to-speech', 'text2text-generation', 'token-classification', 'translation', 'video-classification', 'visual-question-answering', 'vqa', 'zero-shot-audio-classification', 'zero-shot-classification', 'zero-shot-image-classification', 'zero-shot-object-detection', 'translation_XX_to_YY']"
"""

nlp = pipeline("ner", model=model, tokenizer=tokenizer)

# Provide the text you want to analyze and a list of possible entities
text = "John Smith went to Washington D.C. to visit the White House."
entities = ["PERSON", "LOCATION", "ORGANIZATION"]

# Perform NER
#outputs = nlp(text, entities=entities)
outputs = nlp(text)

# Print the results
print(outputs)

Some weights of T5ForTokenClassification were not initialized from the model checkpoint at t5-small and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'entity': 'LABEL_0', 'score': 0.9933555, 'index': 0, 'word': '▁John', 'start': 0, 'end': 4}, {'entity': 'LABEL_0', 'score': 0.99628663, 'index': 1, 'word': '▁Smith', 'start': 5, 'end': 10}, {'entity': 'LABEL_0', 'score': 0.99956733, 'index': 2, 'word': '▁went', 'start': 11, 'end': 15}, {'entity': 'LABEL_0', 'score': 0.99939275, 'index': 3, 'word': '▁to', 'start': 16, 'end': 18}, {'entity': 'LABEL_0', 'score': 0.9988667, 'index': 4, 'word': '▁Washington', 'start': 19, 'end': 29}, {'entity': 'LABEL_0', 'score': 0.97926426, 'index': 5, 'word': '▁D', 'start': 30, 'end': 31}, {'entity': 'LABEL_0', 'score': 0.9942742, 'index': 6, 'word': '.', 'start': 31, 'end': 32}, {'entity': 'LABEL_0', 'score': 0.88043576, 'index': 7, 'word': 'C', 'start': 32, 'end': 33}, {'entity': 'LABEL_0', 'score': 0.6050329, 'index': 8, 'word': '.', 'start': 33, 'end': 34}, {'entity': 'LABEL_0', 'score': 0.5871554, 'index': 9, 'word': '▁to', 'start': 35, 'end': 37}, {'entity': 'LABEL_0', 'score': 0.82891214, 'index